# Implementing an untyped Lambda Calculus


To introduce `metadsl` we will show how to create a working version of the untyped lambda calculus.

This will let us create the some basic combinators, verify that they before as expected under equiavelency.

Then we can also create an implementation of arithmetic on natural numbers, using church encoding and show how to convert
between these and Python's integers.


## Terms
The lambda calculus might sound abstract and odd, but if you know how to write functions in Python you already have mastered it!
The idea is a very simple language that only has functions that take in one argument.
Nothing, else, which seems a bit silly maybe, and it is not very useful by itself, but when it is integrated into a system with other data
types, it becomes a useful so that we can represent functions.


First, let's create a concept of a "variable" that can optionally have a name. It should only be equal to itself, so that we don't have to worry about shadowing variable names:

In [1]:
import dataclasses
import typing


@dataclasses.dataclass(eq=False)
class Variable:
    name: typing.Optional[str] = None
    
    
    def __str__(self):
        return self.name or str(id(self))
        
    def __repr__(self):
        return f"Variable({str(self)})"
        
x1 = Variable("x")
x2 = Variable("x")

assert x1 == x1
assert x1 != x2
assert hash(x1) != hash(x2)

Now, let's build our one and only key abstraction, a lambda term.
You should be able to call it and also create it by having a body and a variable. 

In [2]:
import metadsl


class LambdaTerm(metadsl.Instance):
    @metadsl.call(lambda self, arg: LambdaTerm)
    def __call__(self, arg: "LambdaTerm") -> "LambdaTerm":
        ...

@metadsl.call(lambda variable, body: LambdaTerm)
def abstraction(variable: Variable, body: LambdaTerm) -> LambdaTerm:
    ...

@metadsl.call(lambda variable: LambdaTerm)
def from_variable(variable: Variable) -> LambdaTerm:
    ...

def create_abstraction(fn: typing.Callable[[LambdaTerm], LambdaTerm]) -> LambdaTerm:
    arg = Variable()
    return abstraction(arg, fn(from_variable(arg)))

We can construct the identity function

In [3]:
@create_abstraction
def identity(x):
    return x

In [4]:
identity

LambdaTerm(_call=Call(abstraction, (Variable(4398377392), LambdaTerm(_call=Call(from_variable, (Variable(4398377392),))))))

and then call it on a term:

In [5]:
y_term = from_variable(Variable("y"))
called_identity = identity(y_term)
called_identity

LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(abstraction, (Variable(4398377392), LambdaTerm(_call=Call(from_variable, (Variable(4398377392),)))))), LambdaTerm(_call=Call(from_variable, (Variable(y),))))))

## Pretty Printing

This is a bit of a verbose way to print these.

Let's create a custom function that walks the tree and converts them to a string:

In [6]:
pretty_print = metadsl.Replacements()


@metadsl.call(lambda s: LambdaTerm)
def string_term(s: str) -> LambdaTerm:
    ...


@pretty_print.register(None)
def pp_variable(variable: Variable):
    return (
        from_variable(variable),
        lambda: string_term(str(variable))
    )

@pretty_print.register(None, None)
def pp_abstraction(variable: Variable, body: str):
    return (
        abstraction(variable, string_term(body)),
        lambda: string_term(f"(λ{variable}.{body})")
    )

@pretty_print.register(None, None)
def pp_application(fn: str, arg: str):
    return (
        string_term(fn)(string_term(arg)),
        lambda: string_term(f"({fn} {arg})"),
    )

@metadsl.call(lambda t: pretty_print)
def unbox_str(t: LambdaTerm) -> str:
    ...

@pretty_print.register_pure(None)
def pp_unbox(s: str):
    return unbox_str(string_term(s)), s

In [7]:
unbox_str(called_identity)

'((λ4398377392.4398377392) y)'

## Beta Reduction

To make this useful, we need to define some semantics on these.

What's a useful thing to do with the lambda calculus?

Well it would be nice to actually be able to "compute" these values by replacing all instances of a variable with it's argument.

So let's replace all instances of calling a variable by replacing the arg inside of it:

In [12]:
beta_reduce_replacements = metadsl.Replacements()


@metadsl.call(lambda t: beta_reduce_replacements)
def beta_reduce(t: LambdaTerm) -> LambdaTerm:
    ...

@beta_reduce_replacements.register_pure(LambdaTerm)
def _remove_beta_reduce(t: LambdaTerm):
    return beta_reduce(t), t
    
@beta_reduce_replacements.register(None, LambdaTerm, LambdaTerm)
def _beta_reduce(var: Variable, body: LambdaTerm, arg: LambdaTerm):
    def replacement(var=var, body=body, arg=arg):
        # replaces all instances of var with arg
        rs = metadsl.Replacements()
        rs.register_pure()(lambda: (from_variable(var), arg))
        return rs(body)

    return (
        abstraction(var, body)(arg),
        replacement
    )
beta_reduce(called_identity)

LambdaTerm(_call=Call(from_variable, (Variable(y),)))

Now let's combine these two, to execute them in sequence. First we want to beta reduce, until there is no more to beta reduce, then
we wanna turn that into a pretty print representation:

In [13]:
def execute(expr: LambdaTerm) -> str:
    return unbox_str(beta_reduce_replacements(expr))

In [14]:
print(execute(called_identity))

y


## Alpha Conversion

This looks allright, but before we do anything serious, let's deal with the annoying issue of these variable names. They aren't much fun to read! Let's fix this by implementing [α-conversion](https://en.wikipedia.org/wiki/Lambda_calculus#%CE%B1-conversion). The basic idea is that given some lambda expression, we rename all the variables to make them standard. Let's create a bunch of global variables, and then create a function that will transform an expression into a normal form using them:

In [17]:
GLOBAL_VARS = [Variable(chr(i)) for i in range(ord('a'), ord('a') + 100)]
CURRENT_GLOBAL_VAR = 0

In [18]:
GLOBAL_VARS[0]

Variable(a)

The idea is that we will map each variable we find in the expression to one of the global variables:

In [19]:
alpha_convert_replacements = metadsl.Replacements()

def execute_alpha_convert(x):
    global CURRENT_GLOBAL_VAR
    CURRENT_GLOBAL_VAR = 0
    return alpha_convert_replacements(x)

@metadsl.call(lambda t: execute_alpha_convert)
def alpha_convert(t: LambdaTerm) -> LambdaTerm:
    ...

@alpha_convert_replacements.register_pure(LambdaTerm)
def _remove_alpha_convert(t: LambdaTerm):
    return alpha_convert(t), t


@alpha_convert_replacements.register(None, LambdaTerm)
def _alpha_convert(var: Variable, body: LambdaTerm):
    def replace(var=var, body=body):
        if var in GLOBAL_VARS: 
            return None
        global CURRENT_GLOBAL_VAR

        new_var = GLOBAL_VARS[CURRENT_GLOBAL_VAR]
        CURRENT_GLOBAL_VAR += 1

        rs = metadsl.Replacements()
        rs.register_pure()(lambda: (var, new_var))
  
        res = abstraction(new_var, rs(body))
        
        # We have already replaced this if we got the same
        if res == abstraction(var, body):
            return None
        return res
    
    return (
        abstraction(var, body),
        replace
    )

In [20]:
identity

LambdaTerm(_call=Call(abstraction, (Variable(4398377392), LambdaTerm(_call=Call(from_variable, (Variable(4398377392),))))))

In [21]:
alpha_convert(identity)

LambdaTerm(_call=Call(abstraction, (Variable(a), LambdaTerm(_call=Call(from_variable, (Variable(a),))))))

Now let's add this to our execution step, so that we print before and after beta reducing:

In [22]:
core_replacements = metadsl.ReplacementsSequence(beta_reduce)

def execute(expr: LambdaTerm) -> str:
    expr = alpha_convert(expr)
    return f"{unbox_str(expr)} ⇒ {unbox_str(core_replacements(expr))}"


In [23]:
execute(identity(identity))

'((λa.a) (λb.b)) ⇒ (λb.b)'

Now we are finally getting somewhere!

In [24]:
execute(identity(identity)(from_variable(Variable("Input"))))

'(((λa.a) (λb.b)) Input) ⇒ Input'

Let's actually just set it to always pretty print this representation:

In [25]:
text_formatter = get_ipython().display_formatter.formatters['text/html']
text_formatter.for_type(LambdaTerm, lambda e, *args: execute(e))

In [26]:
identity

LambdaTerm(_call=Call(abstraction, (Variable(4398377392), LambdaTerm(_call=Call(from_variable, (Variable(4398377392),))))))

In [27]:
identity(identity)

LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(abstraction, (Variable(4398377392), LambdaTerm(_call=Call(from_variable, (Variable(4398377392),)))))), LambdaTerm(_call=Call(abstraction, (Variable(4398377392), LambdaTerm(_call=Call(from_variable, (Variable(4398377392),)))))))))

## Church Numerals

Now that we have a working system, we can start to implement some more useful programming on top of it.

One thing we can do is [represent natural numbers](https://en.wikipedia.org/wiki/Church_encoding#Church_numerals) with lambda expression and do math on them.

### Python Ints
First, let's add the ability to embed Python's ints inside a function:

In [28]:
@metadsl.call(lambda i: LambdaTerm)
def int_term(i: int) -> LambdaTerm:
    ...

In [29]:
int_term(123)

LambdaTerm(_call=Call(int_term, (123,)))

Oops! We don't know how to turn a int represenation of a lambda term into a string, lets fix that:

In [30]:
@pretty_print.register(None)
def int_to_str(i: int):
    return int_term(i), lambda: string_term(str(i))

In [31]:
identity(int_term(123))

LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(abstraction, (Variable(4398377392), LambdaTerm(_call=Call(from_variable, (Variable(4398377392),)))))), LambdaTerm(_call=Call(int_term, (123,))))))

### Church Numerals

How do we create a church numeral? Well we have a function that calls itself $n$ times, to represent $n$:

In [32]:
def church_numeral(n: int) -> LambdaTerm:
    @create_abstraction
    def outer(f):
        
        @create_abstraction
        def inner(x):
            for i in range(n):
                x = f(x)
            return x
        return inner
    return outer

In [33]:
church_numeral(0)

LambdaTerm(_call=Call(abstraction, (Variable(4405644256), LambdaTerm(_call=Call(abstraction, (Variable(4405673040), LambdaTerm(_call=Call(from_variable, (Variable(4405673040),)))))))))

In [34]:
x = church_numeral(1)
x

LambdaTerm(_call=Call(abstraction, (Variable(4405684768), LambdaTerm(_call=Call(abstraction, (Variable(4405683928), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4405684768),))), LambdaTerm(_call=Call(from_variable, (Variable(4405683928),))))))))))))

In [35]:
church_numeral(2)

LambdaTerm(_call=Call(abstraction, (Variable(4408697576), LambdaTerm(_call=Call(abstraction, (Variable(4405715856), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4408697576),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4408697576),))), LambdaTerm(_call=Call(from_variable, (Variable(4405715856),)))))))))))))))

In [36]:
church_numeral(3)

LambdaTerm(_call=Call(abstraction, (Variable(4407119544), LambdaTerm(_call=Call(abstraction, (Variable(4407117304), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4407119544),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4407119544),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4407119544),))), LambdaTerm(_call=Call(from_variable, (Variable(4407117304),))))))))))))))))))

Now let's add some replacements that from python integers to church numerals:

In [37]:
to_church_replacements = metadsl.Replacements()

@to_church_replacements.register(None)
def int_to_abstraction(i: int):
    return int_term(i), lambda: church_numeral(i)

In [38]:
to_church_replacements(int_term(10))

LambdaTerm(_call=Call(abstraction, (Variable(4406154464), LambdaTerm(_call=Call(abstraction, (Variable(4406156032), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4406154464),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4406154464),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4406154464),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4406154464),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4406154464),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4406154464),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4406154464),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4406154464),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4406154464),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4406154464),))), LambdaTerm(_call=Call(from_variable, (Variable(4406156032),)))))))))))))))))))))))))))))))))))))))

### Math

Add finally we can define math on these church numerals:

In [39]:
@metadsl.call(lambda x, y: LambdaTerm)
def add(x: LambdaTerm, y: LambdaTerm) -> LambdaTerm:
    ...

@metadsl.call(lambda x: LambdaTerm)
def succ(x: LambdaTerm) -> LambdaTerm:
    ...

@metadsl.call(lambda x, y: LambdaTerm)
def mult(x: LambdaTerm, y: LambdaTerm) -> LambdaTerm:
    ...

@metadsl.call(lambda x, y: LambdaTerm)
def exp(x: LambdaTerm, y: LambdaTerm) -> LambdaTerm:
    ...

#### on Python Integers
First, we can define how these functions behave on Python integers:

In [40]:
int_math_replacements = metadsl.Replacements()

@int_math_replacements.register(None, None)
def _add_ints(x: int, y: int):
    return add(int_term(x), int_term(y)), lambda: int_term(x + y)

@int_math_replacements.register(None)
def _succ_int(x: int):
    return succ(int_term(x)), lambda: int_term(x + 1)

@int_math_replacements.register(None, None)
def _mult_int(x: int, y: int):
    return mult(int_term(x), int_term(y)), lambda: int_term(x * y)


@int_math_replacements.register(None, None)
def _exp_int(x: int, y: int):
    return exp(int_term(x), int_term(y)), lambda: int_term(x ** y)

Now we can add these to our core replacements that we defined above and our executed during printing:

In [41]:
core_replacements.register(int_math_replacements)

In [42]:
add(int_term(1), int_term(2))

LambdaTerm(_call=Call(add, (LambdaTerm(_call=Call(int_term, (1,))), LambdaTerm(_call=Call(int_term, (2,))))))

Oh, we forgot to define the pretty printing version of these operations. Let's do that now as well:

In [43]:
@pretty_print.register(None, None)
def _add_str(x: str, y: str):
    return add(string_term(x), string_term(y)), lambda: string_term(f"{x} + {y}")

@pretty_print.register(None)
def _succ_str(x: str):
    return succ(string_term(x)), lambda: string_term(f"succ({x})")

@pretty_print.register(None, None)
def _mult_str(x: str, y: str):
    return mult(string_term(x), string_term(y)), lambda: string_term(f"{x} * {y}")

@pretty_print.register(None, None)
def _exp_str(x: str, y: str):
    return exp(string_term(x), string_term(y)), lambda: string_term(f"{x}^{y}")


In [44]:
add(int_term(1), int_term(2))

LambdaTerm(_call=Call(add, (LambdaTerm(_call=Call(int_term, (1,))), LambdaTerm(_call=Call(int_term, (2,))))))

Sweet!

#### on Church numerals
But what if we turn them into church numerals first?

In [45]:
to_church_replacements(add(int_term(1), int_term(2)))

LambdaTerm(_call=Call(add, (LambdaTerm(_call=Call(abstraction, (Variable(4406406392), LambdaTerm(_call=Call(abstraction, (Variable(4406407008), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4406406392),))), LambdaTerm(_call=Call(from_variable, (Variable(4406407008),)))))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4406406448), LambdaTerm(_call=Call(abstraction, (Variable(4406406560), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4406406448),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4406406448),))), LambdaTerm(_call=Call(from_variable, (Variable(4406406560),))))))))))))))))))

Well we don't know how to add those. First, let's define the operations as abstractions themselves (following wikipedia here on the naming):

In [73]:
@create_abstraction
def add_abstraction(m):
    @create_abstraction
    def inner(n):
        @create_abstraction
        def inner(f):
            @create_abstraction
            def inner(x):
                return m(f)(n(f)(x))
            return inner
        return inner
    return inner

@create_abstraction
def succ_abstraction(n):
    @create_abstraction
    def inner(f):
        @create_abstraction
        def inner(x):
            return f(n(f)(x))
        return inner
    return inner

@create_abstraction
def mult_abstraction(m):
    @create_abstraction
    def inner(n):
        @create_abstraction
        def inner(f):
            return m(n(f))
        return inner
    return inner

@create_abstraction
def exp_abstraction(m):
    @create_abstraction
    def inner(n):
        return n(m)
    return inner


Is this right? Well let's try it out!

In [74]:
zero = to_church_replacements(int_term(0))
zero

LambdaTerm(_call=Call(abstraction, (Variable(4403616960), LambdaTerm(_call=Call(abstraction, (Variable(4403617520), LambdaTerm(_call=Call(from_variable, (Variable(4403617520),)))))))))

In [75]:
add_abstraction(zero)(zero)

LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(abstraction, (Variable(4404698360), LambdaTerm(_call=Call(abstraction, (Variable(4404698192), LambdaTerm(_call=Call(abstraction, (Variable(4404698136), LambdaTerm(_call=Call(abstraction, (Variable(4404698640), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4404698360),))), LambdaTerm(_call=Call(from_variable, (Variable(4404698136),)))))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4404698192),))), LambdaTerm(_call=Call(from_variable, (Variable(4404698136),)))))), LambdaTerm(_call=Call(from_variable, (Variable(4404698640),))))))))))))))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4403616960), LambdaTerm(_call=Call(abstraction, (Variable(4403617520), LambdaTerm(_call=Call(from_variable, (Variable(4403617520),)))))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4403616960), LambdaTerm(_call=Call(abstraction, (Variable(4403617520), LambdaTerm(_call=Call(from_variable, (Variable(4403617520),))))))))))))

In [76]:
zero

LambdaTerm(_call=Call(abstraction, (Variable(4403616960), LambdaTerm(_call=Call(abstraction, (Variable(4403617520), LambdaTerm(_call=Call(from_variable, (Variable(4403617520),)))))))))

Sweet! Those look same, that's good. 

And we should find that adding one is the same as the succesor functions...

In [82]:
one = to_church_replacements(int_term(1))

In [83]:
add_abstraction(one)

LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(abstraction, (Variable(4404698360), LambdaTerm(_call=Call(abstraction, (Variable(4404698192), LambdaTerm(_call=Call(abstraction, (Variable(4404698136), LambdaTerm(_call=Call(abstraction, (Variable(4404698640), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4404698360),))), LambdaTerm(_call=Call(from_variable, (Variable(4404698136),)))))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4404698192),))), LambdaTerm(_call=Call(from_variable, (Variable(4404698136),)))))), LambdaTerm(_call=Call(from_variable, (Variable(4404698640),))))))))))))))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4403720880), LambdaTerm(_call=Call(abstraction, (Variable(4403720544), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4403720880),))), LambdaTerm(_call=Call(from_variable, (Variable(4403720544),)))))))))))))))

In [78]:
succ_abstraction

LambdaTerm(_call=Call(abstraction, (Variable(4404697352), LambdaTerm(_call=Call(abstraction, (Variable(4404697464), LambdaTerm(_call=Call(abstraction, (Variable(4404658360), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4404697464),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4404697352),))), LambdaTerm(_call=Call(from_variable, (Variable(4404697464),)))))), LambdaTerm(_call=Call(from_variable, (Variable(4404658360),))))))))))))))))))

Those look the same! Nice!

OK What about multiplying by zero, that should always return zero...

In [79]:
mult_abstraction(zero)(from_variable(Variable("doesnt matter")))

LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(abstraction, (Variable(4404724904), LambdaTerm(_call=Call(abstraction, (Variable(4404724680), LambdaTerm(_call=Call(abstraction, (Variable(4404724568), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4404724904),))), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4404724680),))), LambdaTerm(_call=Call(from_variable, (Variable(4404724568),)))))))))))))))))), LambdaTerm(_call=Call(abstraction, (Variable(4403616960), LambdaTerm(_call=Call(abstraction, (Variable(4403617520), LambdaTerm(_call=Call(from_variable, (Variable(4403617520),)))))))))))), LambdaTerm(_call=Call(from_variable, (Variable(doesnt matter),))))))

Yep!

It's kind cool how exponentiation is now so simple... Let's try raising something to the 1st power, this should give us back itself:

In [85]:
exp_abstraction

LambdaTerm(_call=Call(abstraction, (Variable(4404724288), LambdaTerm(_call=Call(abstraction, (Variable(4404724960), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4404724960),))), LambdaTerm(_call=Call(from_variable, (Variable(4404724288),))))))))))))

In [86]:
exp_abstraction(from_variable(Variable("doesnt matter")))(one)

LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(abstraction, (Variable(4404724288), LambdaTerm(_call=Call(abstraction, (Variable(4404724960), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4404724960),))), LambdaTerm(_call=Call(from_variable, (Variable(4404724288),)))))))))))), LambdaTerm(_call=Call(from_variable, (Variable(doesnt matter),)))))), LambdaTerm(_call=Call(abstraction, (Variable(4403720880), LambdaTerm(_call=Call(abstraction, (Variable(4403720544), LambdaTerm(_call=Call(__call__, (LambdaTerm(_call=Call(from_variable, (Variable(4403720880),))), LambdaTerm(_call=Call(from_variable, (Variable(4403720544),)))))))))))))))

This is actually to the input, despite it different looking output. If we had implemented [Eta-conversion](https://en.wikipedia.org/wiki/Lambda_calculus#%CE%B7-conversion) this would reduce to "doesn't matter".